# Problem 1:

Newton Raphson method is outlined in the following snippet:

In [51]:
import pandas as pd
import numpy as np

def newton_raphson(x0, bu_0, max_iterations, dataset_name):
    """
    Function description: perform newton raphson to compute position of observer based on available satellite
    data, stop function when we have updates in delta_x, delta_bu that are less than a centimeter,
    or hit maximum number of iterations

    Inputs:
    1) x_0: initial guess for position
    2) bu_0: initial guess for clock bias
    3) max_iterations: maximum number of iterations until covergence
    4) dataset_name: name of file to read in

    return: 
    1) optimized position
    2) optimized clock bias
    3) number of iterations until covergence
    """
    x_est = x0
    b_est = bu_0

    dataset = pd.read_csv(dataset_name)

    for i in range(max_iterations):

        # Get Geometry Matrix and deltas for rho
        geometry_matrix = []
        delta_rho = []
        for index, sat in dataset.iterrows():
            geometry_matrix.append(get_geometry_row(x_est, sat['x_sv_m'], sat['y_sv_m'], sat['z_sv_m']))
            delta_rho.append(sat['corr_pr_m'] - get_theoretical_pseudoranges(x_est, b_est, sat['x_sv_m'], sat['y_sv_m'], sat['z_sv_m']))

        # Convert into matricies to make it easier to use numpy functions
        geometry_matrix = np.array(geometry_matrix)
        delta_rho = np.array(delta_rho)

        # Update the deltas
        delta_x, delta_bu = newton_raphson_step(geometry_matrix, delta_rho)
        x_est = np.add(x_est, delta_x)
        b_est = b_est + delta_bu
    
        if True: #i%10 == 0:
            print("round: ",i+1)
            print("x_est: ", x_est)
            print("b_est: ", b_est)

        if delta_x[0] < 0.01 and delta_x[1] < 0.01 and delta_x[2] < 0.01 and delta_bu < 0.01:
            break
    
    
    print("round: ",i+1)
    print("FINAL - x_est: ", x_est)
    print("FINAL - b_est: ", b_est)
    return x_est


def get_geometry_row(x_est, x_sv_m, y_sv_m, z_sv_m):
    """
    Function description: Compute each row for the geometry matrix given a satellite position and
    current guess for position estimate

    Inputs:
    1) x_est: current guess for current position estimate 
    2) x_sv_m: a set for satellite position x
    3) y_sv_m: a set for satellite position y
    4) z_sv_m: a set for satellite position z


    return: 
    1) Geometry Matrix G (first 3 columns unit vector)
    """

    x = x_sv_m - x_est[0]
    y = y_sv_m - x_est[1]
    z = z_sv_m - x_est[2]

    magnitude = np.sqrt((x)**2 + (y)**2 + (z)**2)

    return ([-x/magnitude, -y/magnitude, -z/magnitude, 1])



def get_theoretical_pseudoranges(x_est, b_est, x_sv_m, y_sv_m, z_sv_m):
    """
    Function description: Compute each row for the geometry matrix given a satellite position and
    current guess for position estimate

    Inputs:
    1) x_est: current position estimate 
    2) b_est: current clock bias estimate 
    3) x_sv_m: a set for satellite position x
    4) y_sv_m: a set for satellite position y
    5) z_sv_m: a set for satellite position z

    return: 
    1) Theoretical pseudorange measurement vector
    """


    x = x_sv_m - x_est[0]
    y = y_sv_m - x_est[1]
    z = z_sv_m - x_est[2]

    magnitude = np.sqrt((x)**2 + (y)**2 + (z)**2)
    b_k = 0 # assume its zero for all satellites, already corrrected for in data

    return magnitude + b_est - b_k

def newton_raphson_step(G,delta_rho):
    """
    Function description: Take one step of newton-raphson optimization, compute update of 
    delta_x and delta_bu

    Input:
    1) G: geometry matrix
    2) delta_rho: difference of theoretical and measured pseudoranges 

    return:
    1) delta_x
    2) delta_bu
    """
    G_t = G.transpose()
    deltas = np.matmul(np.matmul(np.linalg.inv(np.matmul(G_t, G)),G_t),delta_rho)

    delta_x = deltas[0:3]
    delta_bu = deltas[3]

    return delta_x, delta_bu



In [56]:
# hw2_dataset/hw2_p1_dataset1_navdata.csv
# hw2_dataset/hw2_p1_dataset2_navdata.csv
# hw2_dataset/hw2_p1_dataset3_navdata.csv

import gnss_lib_py as glp
x = newton_raphson([2694627.03315213, 4296768.45811191 , -3854472.8087874 ],0,30,"hw2_dataset/hw2_p1_dataset3_navdata.csv")

print("Lat/Long Version: ",glp.ecef_to_geodetic(x))

round:  1
x_est:  [-4552912.82652256 -7012102.03939962  6772201.37607263]
b_est:  4578637.186411848
round:  2
x_est:  [-2819040.93352791 -4466289.79537935  4083554.34443848]
b_est:  397150.018145015
round:  3
x_est:  [-2695297.00293215 -4297676.40793132  3855932.57672542]
b_est:  1880.6077536040102
round:  4
x_est:  [-2694627.05387585 -4296768.48647448  3854472.86292769]
b_est:  -272.3767554751271
round:  5
x_est:  [-2694627.03315213 -4296768.45811191  3854472.80878741]
b_est:  -272.4524555035465
round:  6
x_est:  [-2694627.03315213 -4296768.45811191  3854472.8087874 ]
b_est:  -272.45245550738935
round:  6
FINAL - x_est:  [-2694627.03315213 -4296768.45811191  3854472.8087874 ]
FINAL - b_est:  -272.45245550738935
Lat/Long Version:  [[  37.419674   -122.09300692  -24.15400905]]


# Problem 3

3.1) function `get_unit_vector`

3.3) function `compute_dop_matrix`

3.4) function `compute_pdop`

In [45]:
import numpy as np 
def get_unit_vector(az, el):
    """
    Function description: Give unit vector in ENU direction vector from satellite az and el
    
    Inputs:
    1) az: satellite Azimuth
    2) el: satellite elevation

    return:
    1) 3 dimensional unit vector  that represents satellite in ENU reference frame
    """

    E = np.sin(np.deg2rad(az)) * np.cos(np.deg2rad(el))
    N = np.cos(np.deg2rad(az)) * np.cos(np.deg2rad(el))
    U = np.sin(np.deg2rad(el))

    r = np.sqrt(E**2+N**2+U**2)

    return E* r,N * r,U * r

def compute_dop_matrix(az_arr, el_arr):
    """
    Function description: Give a DOP matrix given array of satellite az and el angles

    Inputs:
    1) az_arr: satellite Azimuth array
    2) el_arr: satellite elevation array

    return:
    1) DOP matrix
    """

    # Get Geometry Matrix and deltas for rho
    enut_matrix = []
    for az, el in zip(az_arr,el_arr):
        E,N,U = get_unit_vector(az, el)
        enut_matrix.append([E,N,U,1])

    enut_matrix = np.array(enut_matrix)
        
    enut_matrix_t = enut_matrix.transpose()
    H = np.linalg.inv(np.matmul(enut_matrix_t, enut_matrix))
    return H

def compute_pdop(H):
    """
    Function description: Output PDOP from DOP matrix

    Inputs:
    1) H: DOP matrix

    return:
    1) PDOP
    """

    # Get Geometry Matrix and deltas for rho
    pdop = np.sqrt(H[0,0]+H[1,1]+H[2,2])
    return pdop


        

In [46]:
get_unit_vector(203.008351,6.707822)

(-0.38818970486176474, -0.9141471631365393, 0.11680632332520811)

In [54]:

dataset = pd.read_csv("hw2_dataset/hw2_p3_az_el_data_navdata.csv")
az_arr = dataset['az_sv_deg']
el_arr = dataset['el_sv_deg']
H = compute_dop_matrix(az_arr, el_arr)
print(H)
compute_pdop(H)

[[ 0.20097502 -0.01913473  0.05142934 -0.03780211]
 [-0.01913473  0.43919681 -0.19899284  0.14190813]
 [ 0.05142934 -0.19899284  1.03224535 -0.48714849]
 [-0.03780211  0.14190813 -0.48714849  0.32733601]]


1.2932196947575985

### Question 3.5 Urban Canyon

In [53]:

data_new = dataset[dataset['sv_id'].isin([3,2,21,28])]
az_arr = data_new['az_sv_deg']
el_arr = data_new['el_sv_deg']
H = compute_dop_matrix(az_arr, el_arr)
print(H)
compute_pdop(H)

[[16.6282074  22.82428202 -6.8016688   7.21293016]
 [22.82428202 34.41260325 -7.66052989  8.40886355]
 [-6.8016688  -7.66052989  7.03771467 -5.87671075]
 [ 7.21293016  8.40886355 -5.87671075  5.43863782]]


7.62092680198727

,gps_millis,gnss_sv_id,gnss_id,sv_id,x_sv_m,y_sv_m,z_sv_m,el_sv_deg,az_sv_deg
0,1.409515e+12,G02,gps,2,-1.694697e+07,-3.160271e+06,2.076307e+07,44.594897,306.600560
1,1.409515e+12,G03,gps,3,-2.057177e+07,9.520604e+06,1.358305e+07,9.815427,291.910824
2,1.409515e+12,G08,gps,8,-2.583507e+07,-6.493357e+06,-3.020425e+06,17.022331,232.570970
3,1.409515e+12,G10,gps,10,7.110526e+06,-2.161120e+07,1.362991e+07,37.254877,83.371335
4,1.409515e+12,G21,gps,21,-1.795699e+07,-9.813602e+06,1.788139e+07,60.635433,288.647324
5,1.409515e+12,G23,gps,23,1.622704e+07,-2.082972e+07,1.384407e+06,3.633839,99.901248
6,1.409515e+12,G25,gps,25,1.979932e+07,-1.613370e+07,6.439062e+06,0.009051,82.691649
7,1.409515e+12,G27,gps,27,-1.980194e+07,-1.295897e+07,-1.276641e+07,6.707822,203.008351
8,1.409515e+12,G28,gps,28,-3.797052e+06,-2.520792e+07,7.441063e+06,51.799560,128.938586
9,1.409515e+12,G31,gps,31,-1.086848e+07,-2.356794e+07,-4.806161e+06,29.479175,170.179292
